# 继承collections.abc以实现自定义的容器类型

In [12]:
import logging

**示例：**创建一种自定义的列表类型，并提供统计各元素出现的频率的方法。

In [2]:
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
    
    def frequency(self):
        counts = {}
        for item in self:
            counts.setdefault(item, 0)
            counts[item] += 1
        return counts

In [5]:
foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])
print('Length is', len(foo))
foo.pop()
print('After pop:', repr(foo))
print('Frequency:', foo.frequency())

Length is 7
After pop: ['a', 'b', 'a', 'c', 'b', 'a']
Frequency: {'a': 3, 'b': 2, 'c': 1}


**需求更改：**编写一种对象，本身虽然不属于list子类，但是用起来却和list一样，也可以通过下标访问其中元素。

**示例：**令下面这个表示二叉树节点的类，也能够像list或tuple等序列那样来访问。

In [6]:
class BinaryNode(object):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

In [7]:
bar = [1, 2, 3]
assert bar[0] == bar.__getitem__(0)

通过上面的测试，可以提供自己定制的__getitem__方法，令BinaryNode类可以表现得和序列一样。

In [8]:
class IndexableNode(BinaryNode):
    def _search(self, count, index):
        found = None
        if self.left:
            found, count = self.left._search(count, index)
        if not found and count == index:
            found = self
        else:
            count += 1
        if not found and self.right:
            found, count = self.right._search(count, index)
        # Returns (found, count)
        return found, count      

    def __getitem__(self, index):
        found, _ = self._search(0, index)
        if not found:
            raise IndexError('Index out of range')
        return found.value

In [9]:
tree = IndexableNode(
    10,
    left=IndexableNode(
        5,
        left=IndexableNode(2),
        right=IndexableNode(
            6, right=IndexableNode(7))),
    right=IndexableNode(
        15, left=IndexableNode(11)))

In [10]:
print('LRR =', tree.left.right.right.value)
print('Index 0 =', tree[0])
print('Index 1 =', tree[1])
print('11 in the tree?', 11 in tree)
print('17 in the tree?', 17 in tree)
print('Tree is', list(tree))

LRR = 7
Index 0 = 2
Index 1 = 5
11 in the tree? True
17 in the tree? False
Tree is [2, 5, 6, 7, 10, 11, 15]


In [13]:
try:
    len(tree)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-13-ad92e75b36b3>", line 2, in <module>
    len(tree)
TypeError: object of type 'IndexableNode' has no len()


要使内置的len函数正常运作，必须在自己定制的序列类型中实现__len__的特殊方法。

In [14]:
class SequenceNode(IndexableNode):
    def __len__(self):
        _, count = self._search(0, None)
        return count

In [15]:
tree = SequenceNode(
    10,
    left=SequenceNode(
        5,
        left=SequenceNode(2),
        right=SequenceNode(
            6, right=SequenceNode(7))),
    right=SequenceNode(
        15, left=SequenceNode(11))
)

print('Tree has %d nodes' % len(tree))

Tree has 7 nodes


**改进：**需要提供count和index方法，需要定义自己的容器类型。

In [16]:
try:
    from collections.abc import Sequence
    
    class BadType(Sequence):
        pass
    
    foo = BadType()
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-16-09fffee2c681>", line 7, in <module>
    foo = BadType()
TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__


In [17]:
class BetterNode(SequenceNode, Sequence):
    pass

tree = BetterNode(
    10,
    left=BetterNode(
        5,
        left=BetterNode(2),
        right=BetterNode(
            6, right=BetterNode(7))),
    right=BetterNode(
        15, left=BetterNode(11))
)

print('Index of 7 is', tree.index(7))
print('Count of 10 is', tree.count(10))

Index of 7 is 3
Count of 10 is 1
